In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6589,2021-01-02T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1090,186,1276,33674,...,2468,NaN,NaN,95500,1231722,820100.0,NaN,20.0,NaN,NaN
6590,2021-01-02T17:00:00,ITA,9,Toscana,43.769231,11.255889,825,139,964,9057,...,3720,NaN,NaN,121415,1898439,1066811.0,NaN,6.0,Positivi diagnosticati solo con test antigenic...,NaN
6591,2021-01-02T17:00:00,ITA,10,Umbria,43.106758,12.388247,270,47,317,3476,...,628,NaN,NaN,29238,506539,253466.0,NaN,1.0,NaN,NaN
6592,2021-01-02T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,76,2,78,335,...,383,NaN,NaN,7297,63032,37893.0,NaN,2.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6589,2021-01-02T17:00:00,19,Sicilia,1090,186,1276,33674,34950,603,734,58082,2468,NaN,NaN,95500,1231722,820100.0,20.0,NaN,NaN
6590,2021-01-02T17:00:00,9,Toscana,825,139,964,9057,10021,185,498,107674,3720,NaN,NaN,121415,1898439,1066811.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
6591,2021-01-02T17:00:00,10,Umbria,270,47,317,3476,3793,-11,50,24817,628,NaN,NaN,29238,506539,253466.0,1.0,NaN,NaN
6592,2021-01-02T17:00:00,2,Valle d'Aosta,76,2,78,335,413,5,7,6501,383,NaN,NaN,7297,63032,37893.0,2.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-01-02', '2021-01-01')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-02T17:00:00,1,Piemonte,2777,182,2959,21238,24197,-2332,433,167151,7971,NaN,NaN,199319,1690587,988567.0,0.0,NaN,NaN
1,2021-01-02T17:00:00,2,Valle d'Aosta,76,2,78,335,413,5,7,6501,383,NaN,NaN,7297,63032,37893.0,2.0,NaN,NaN
2,2021-01-02T17:00:00,3,Lombardia,3293,491,3784,52226,56010,698,1402,402070,25281,NaN,NaN,483361,4896792,2621183.0,20.0,NaN,NaN
3,2021-01-02T17:00:00,5,Veneto,2668,372,3040,91081,94121,1539,3165,161049,6675,NaN,NaN,261845,3334350,1224449.0,15.0,NaN,NaN
4,2021-01-02T17:00:00,6,Friuli Venezia Giulia,652,63,715,11132,11847,94,359,37567,1687,NaN,NaN,51101,942810,343713.0,4.0,NaN,NaN
5,2021-01-02T17:00:00,7,Liguria,674,66,740,4962,5702,105,203,52475,2907,NaN,NaN,61084,715806,324658.0,5.0,I 162885 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-02T17:00:00,8,Emilia-Romagna,2649,229,2878,55628,58506,1719,2035,109818,7846,NaN,NaN,176170,2588793,1234681.0,17.0,NaN,NaN
7,2021-01-02T17:00:00,9,Toscana,825,139,964,9057,10021,185,498,107674,3720,NaN,NaN,121415,1898439,1066811.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-02T17:00:00,10,Umbria,270,47,317,3476,3793,-11,50,24817,628,NaN,NaN,29238,506539,253466.0,1.0,NaN,NaN
9,2021-01-02T17:00:00,11,Marche,479,65,544,11243,11787,48,285,29221,1594,NaN,NaN,42602,539624,319908.0,4.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-01T17:00:00,1,Piemonte,2784,184,2968,23561,26529,-1728,1058,164397,7960,NaN,NaN,198886,1688316,987563.0,5.0,NaN,NaN
1,2021-01-01T17:00:00,2,Valle d'Aosta,75,1,76,332,408,-3,17,6501,381,NaN,NaN,7290,62831,37570.0,0.0,NaN,NaN
2,2021-01-01T17:00:00,3,Lombardia,3352,487,3839,51473,55312,689,3056,401444,25203,NaN,NaN,481959,4885034,2617001.0,13.0,NaN,NaN
3,2021-01-01T17:00:00,5,Veneto,2657,372,3029,89553,92582,1509,4805,159469,6629,NaN,NaN,258680,3322582,1220624.0,21.0,NaN,NaN
4,2021-01-01T17:00:00,6,Friuli Venezia Giulia,647,63,710,11043,11753,-45,715,37320,1669,NaN,NaN,50742,940153,342937.0,11.0,NaN,NaN
5,2021-01-01T17:00:00,7,Liguria,664,63,727,4870,5597,-23,412,52386,2898,NaN,NaN,60881,714541,324246.0,4.0,I 162362 test antigenici rapidi sono riferiti ...,NaN
6,2021-01-01T17:00:00,8,Emilia-Romagna,2643,231,2874,53913,56787,-559,2629,109546,7808,NaN,NaN,174141,2583512,1232042.0,11.0,NaN,NaN
7,2021-01-01T17:00:00,9,Toscana,810,143,953,8883,9836,158,589,107381,3700,NaN,NaN,120917,1893356,1064848.0,5.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2021-01-01T17:00:00,10,Umbria,254,46,300,3504,3804,27,228,24759,625,NaN,NaN,29188,506256,253369.0,3.0,NaN,NaN
9,2021-01-01T17:00:00,11,Marche,479,61,540,11199,11739,561,693,28997,1581,NaN,NaN,42317,537533,318488.0,6.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2021-01-02T17:00:00,1,Piemonte,2777,182,2959,21238,24197,-2332,433,167151,7971,NaN,NaN,199319,1690587,988567.0,0.0,NaN,NaN
1,2021-01-02T17:00:00,2,Valle d'Aosta,76,2,78,335,413,5,7,6501,383,NaN,NaN,7297,63032,37893.0,2.0,NaN,NaN
2,2021-01-02T17:00:00,3,Lombardia,3293,491,3784,52226,56010,698,1402,402070,25281,NaN,NaN,483361,4896792,2621183.0,20.0,NaN,NaN
3,2021-01-02T17:00:00,4,Trentino-Alto Adige,545,65,610,12210,12820,128,202,37475,1704,0.0,0.0,51999,818332,315473.0,3.0,0,NaN
4,2021-01-02T17:00:00,5,Veneto,2668,372,3040,91081,94121,1539,3165,161049,6675,NaN,NaN,261845,3334350,1224449.0,15.0,NaN,NaN
5,2021-01-02T17:00:00,6,Friuli Venezia Giulia,652,63,715,11132,11847,94,359,37567,1687,NaN,NaN,51101,942810,343713.0,4.0,NaN,NaN
6,2021-01-02T17:00:00,7,Liguria,674,66,740,4962,5702,105,203,52475,2907,NaN,NaN,61084,715806,324658.0,5.0,I 162885 test antigenici rapidi sono riferiti ...,NaN
7,2021-01-02T17:00:00,8,Emilia-Romagna,2649,229,2878,55628,58506,1719,2035,109818,7846,NaN,NaN,176170,2588793,1234681.0,17.0,NaN,NaN
8,2021-01-02T17:00:00,9,Toscana,825,139,964,9057,10021,185,498,107674,3720,NaN,NaN,121415,1898439,1066811.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2021-01-02T17:00:00,10,Umbria,270,47,317,3476,3793,-11,50,24817,628,NaN,NaN,29238,506539,253466.0,1.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2021-01-01T17:00:00,1,Piemonte,2784,184,2968,23561,26529,-1728,...,164397,7960,NaN,NaN,198886,1688316,987563.0,5.0,NaN,NaN
1,1,2021-01-01T17:00:00,2,Valle d'Aosta,75,1,76,332,408,-3,...,6501,381,NaN,NaN,7290,62831,37570.0,0.0,NaN,NaN
2,2,2021-01-01T17:00:00,3,Lombardia,3352,487,3839,51473,55312,689,...,401444,25203,NaN,NaN,481959,4885034,2617001.0,13.0,NaN,NaN
3,19,2021-01-01T17:00:00,4,Trentino-Alto Adige,521,66,587,12105,12692,161,...,37411,1694,0.0,0.0,51797,817030,315114.0,1.0,0,NaN
4,3,2021-01-01T17:00:00,5,Veneto,2657,372,3029,89553,92582,1509,...,159469,6629,NaN,NaN,258680,3322582,1220624.0,21.0,NaN,NaN
5,4,2021-01-01T17:00:00,6,Friuli Venezia Giulia,647,63,710,11043,11753,-45,...,37320,1669,NaN,NaN,50742,940153,342937.0,11.0,NaN,NaN
6,5,2021-01-01T17:00:00,7,Liguria,664,63,727,4870,5597,-23,...,52386,2898,NaN,NaN,60881,714541,324246.0,4.0,I 162362 test antigenici rapidi sono riferiti ...,NaN
7,6,2021-01-01T17:00:00,8,Emilia-Romagna,2643,231,2874,53913,56787,-559,...,109546,7808,NaN,NaN,174141,2583512,1232042.0,11.0,NaN,NaN
8,7,2021-01-01T17:00:00,9,Toscana,810,143,953,8883,9836,158,...,107381,3700,NaN,NaN,120917,1893356,1064848.0,5.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2021-01-01T17:00:00,10,Umbria,254,46,300,3504,3804,27,...,24759,625,NaN,NaN,29188,506256,253369.0,3.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2021-01-02T17:00:00,1,Piemonte,2777,182,2959,21238,24197,-2332,433,...,7971,NaN,NaN,199319,1690587,988567.0,0.0,NaN,NaN,4341375
1,2021-01-02T17:00:00,2,Valle d'Aosta,76,2,78,335,413,5,7,...,383,NaN,NaN,7297,63032,37893.0,2.0,NaN,NaN,125501
2,2021-01-02T17:00:00,3,Lombardia,3293,491,3784,52226,56010,698,1402,...,25281,NaN,NaN,483361,4896792,2621183.0,20.0,NaN,NaN,10103969
3,2021-01-02T17:00:00,4,Trentino-Alto Adige,545,65,610,12210,12820,128,202,...,1704,0.0,0.0,51999,818332,315473.0,3.0,0,NaN,1074819
4,2021-01-02T17:00:00,5,Veneto,2668,372,3040,91081,94121,1539,3165,...,6675,NaN,NaN,261845,3334350,1224449.0,15.0,NaN,NaN,4907704
5,2021-01-02T17:00:00,6,Friuli Venezia Giulia,652,63,715,11132,11847,94,359,...,1687,NaN,NaN,51101,942810,343713.0,4.0,NaN,NaN,1211357
6,2021-01-02T17:00:00,7,Liguria,674,66,740,4962,5702,105,203,...,2907,NaN,NaN,61084,715806,324658.0,5.0,I 162885 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2021-01-02T17:00:00,8,Emilia-Romagna,2649,229,2878,55628,58506,1719,2035,...,7846,NaN,NaN,176170,2588793,1234681.0,17.0,NaN,NaN,4467118
8,2021-01-02T17:00:00,9,Toscana,825,139,964,9057,10021,185,498,...,3720,NaN,NaN,121415,1898439,1066811.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2021-01-02T17:00:00,10,Umbria,270,47,317,3476,3793,-11,50,...,628,NaN,NaN,29238,506539,253466.0,1.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2021-01-02T17:00:00,1,Piemonte,2777,182,2959,21238,24197,-2332,433,...,NaN,NaN,199319,1690587,988567.0,0.0,NaN,NaN,4341375,575
1,2021-01-02T17:00:00,2,Valle d'Aosta,76,2,78,335,413,5,7,...,NaN,NaN,7297,63032,37893.0,2.0,NaN,NaN,125501,20
2,2021-01-02T17:00:00,3,Lombardia,3293,491,3784,52226,56010,698,1402,...,NaN,NaN,483361,4896792,2621183.0,20.0,NaN,NaN,10103969,1036
3,2021-01-02T17:00:00,4,Trentino-Alto Adige,545,65,610,12210,12820,128,202,...,0.0,0.0,51999,818332,315473.0,3.0,0,NaN,1074819,106
4,2021-01-02T17:00:00,5,Veneto,2668,372,3040,91081,94121,1539,3165,...,NaN,NaN,261845,3334350,1224449.0,15.0,NaN,NaN,4907704,1016
5,2021-01-02T17:00:00,6,Friuli Venezia Giulia,652,63,715,11132,11847,94,359,...,NaN,NaN,51101,942810,343713.0,4.0,NaN,NaN,1211357,180
6,2021-01-02T17:00:00,7,Liguria,674,66,740,4962,5702,105,203,...,NaN,NaN,61084,715806,324658.0,5.0,I 162885 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2021-01-02T17:00:00,8,Emilia-Romagna,2649,229,2878,55628,58506,1719,2035,...,NaN,NaN,176170,2588793,1234681.0,17.0,NaN,NaN,4467118,634
8,2021-01-02T17:00:00,9,Toscana,825,139,964,9057,10021,185,498,...,NaN,NaN,121415,1898439,1066811.0,6.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2021-01-02T17:00:00,10,Umbria,270,47,317,3476,3793,-11,50,...,NaN,NaN,29238,506539,253466.0,1.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-01-02T17:00:00,5,Veneto,2668,372,3040,91081,94121,1539,3165,...,36.61,1528,-1640,11768,3825.0,46,1580,26.89,0.06449,5.34
1,2021-01-02T17:00:00,8,Emilia-Romagna,2649,229,2878,55628,58506,1719,2035,...,36.12,1715,-594,5281,2639.0,38,272,38.53,0.04556,3.94
2,2021-01-02T17:00:00,3,Lombardia,3293,491,3784,52226,56010,698,1402,...,47.39,753,-1654,11758,4182.0,78,626,11.92,0.01388,4.78
3,2021-01-02T17:00:00,12,Lazio,2792,315,3107,73341,76448,87,1275,...,37.19,37,-638,9686,4004.0,24,1164,13.16,0.02174,2.83
4,2021-01-02T17:00:00,19,Sicilia,1090,186,1276,33674,34950,603,734,...,31.63,576,-388,5093,3055.0,28,103,14.41,0.01477,1.92
5,2021-01-02T17:00:00,9,Toscana,825,139,964,9057,10021,185,498,...,26.58,174,-91,5083,1963.0,20,293,9.80,0.01338,3.26
6,2021-01-02T17:00:00,1,Piemonte,2777,182,2959,21238,24197,-2332,433,...,31.65,-2323,-625,2271,1004.0,11,2754,19.07,0.00997,4.59
7,2021-01-02T17:00:00,15,Campania,1355,99,1454,76561,78015,-313,392,...,19.60,-304,-1342,3209,1964.0,29,676,12.22,0.00678,3.31
8,2021-01-02T17:00:00,6,Friuli Venezia Giulia,652,63,715,11132,11847,94,359,...,35.00,89,-356,2657,776.0,18,247,13.51,0.02964,4.22
9,2021-01-02T17:00:00,16,Puglia,1503,139,1642,52170,53812,-101,344,...,37.67,-130,-1051,2021,1977.0,10,435,17.02,0.00858,2.31


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,65,41,106,61.32
1,Lombardia,491,545,1036,47.39
2,Marche,65,78,143,45.45
3,Umbria,47,70,117,40.17
4,Puglia,139,230,369,37.67
5,Lazio,315,532,847,37.19
6,Veneto,372,644,1016,36.61
7,Emilia-Romagna,229,405,634,36.12
8,Friuli Venezia Giulia,63,117,180,35.00
9,Piemonte,182,393,575,31.65


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,2020-12-31T17:00:00,22,P.A. Trento,380,42,422,1440,1862,149,320,19036,942,NaN,NaN,21840,450593,151538.0,3.0,NaN,"Dei 320 casi odierni, 150 sono molecolari con ..."
624,2021-01-01T17:00:00,21,P.A. Bolzano,159,27,186,10607,10793,124,270,18229,742,NaN,NaN,29764,363649,163276.0,1.0,NaN,NaN
625,2021-01-01T17:00:00,22,P.A. Trento,362,39,401,1498,1899,37,193,19182,952,NaN,NaN,22033,453381,151838.0,0.0,NaN,"Dei 193 casi odierni, 46 sono molecolari con p..."
626,2021-01-02T17:00:00,21,P.A. Bolzano,176,24,200,10638,10838,45,86,18266,746,NaN,NaN,29850,364372,163511.0,0.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,2020-12-29T17:00:00,4,Trentino-Alto Adige,516,68,584,11854,12438,10,266,...,19.0,2852.0,654.0,18.0,11.64,4.70,9.33,40.67,4.62,0.08
310,2020-12-30T17:00:00,4,Trentino-Alto Adige,548,67,615,11700,12315,-123,417,...,9.0,5003.0,1074.0,-154.0,10.89,4.99,8.33,38.83,-1.47,-0.99
311,2020-12-31T17:00:00,4,Trentino-Alto Adige,539,68,607,11924,12531,216,580,...,17.0,4598.0,957.0,224.0,11.20,4.84,12.61,60.61,1.49,1.75
312,2021-01-01T17:00:00,4,Trentino-Alto Adige,521,66,587,12105,12692,161,463,...,13.0,4978.0,960.0,181.0,11.24,4.62,9.30,48.23,-2.94,1.28


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
